This notebook is for testing the there package.

In [2]:
import sys
import osmnx as ox
import geopandas as gpd
import urbanaccess as ua
import pandas as pd
import there as there

c:\Users\z3258367\Anaconda3\envs\ox_ua_there\lib\site-packages\osmnx\projection.py:3: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [3]:
place = "Adelaide, Australia"
proj_crs = "EPSG:28355"  # GDA94 / MGA zone 55

Having selected a location, we download destination points of interest using OSMnx and the inbuilt POI dictionary. This dictionary can be edited to calculate access to different POIs.

In [4]:
poi_dict = there.get_poi_dict()
place_gdf = ox.geocode_to_gdf(place).to_crs(proj_crs)
osm_pois = there.poi_downloader(place_gdf, poi_dict, proj_crs)

For Australian case studies, employment data is added using ABS census data prepared previously. The basic format required is a location plus a number of jobs available there. If employment data is not available, this can be removed from the indexes.

In [5]:
employment_centrs = gpd.read_file(("example employment data.gpkg")).to_crs(proj_crs)
employment_centrs['category'] = 'employment' #set a category column matching the POI dictionary

In [6]:
all_pois = pd.concat([osm_pois, employment_centrs])
all_pois = gpd.clip(all_pois, place_gdf.to_crs(proj_crs))

c:\Users\z3258367\Anaconda3\envs\ox_ua_there\lib\site-packages\geopandas\array.py:1406: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as GDA94 / MGA zone 55 (the single non-null crs provided).
  warnings.warn(


In [7]:
walk_network = there.prepare_walk_net(place_gdf, proj_crs)

In [8]:
poi_params = there.default_poi_params()
poi_params.loc['employment','num_pois'] = 1200
poi_params.loc['employment','variable'] = "MB Job Count"

## Calculate walking index

In [14]:
maximum_dist = 2400
results_walk = there.there_index(walk_network, all_pois, poi_dict, poi_params['weight'], poi_params['distance_constant'],poi_params['num_pois'],
                             poi_params['diminishing_returns_constant'],poi_variables=poi_params['variable'],distance=maximum_dist)

Finished category: employment
Maximum score: 15.730007015824427 out of 31.9
Finished category: education
Maximum score: 8.375054762147114 out of 14.3
Finished category: shopping
Maximum score: 15.824273051935048 out of 21.6
Finished category: errands
Maximum score: 6.4027665243008975 out of 8.9
Finished category: recreation
Maximum score: 17.22566031440409 out of 23.3


In [32]:
maximum_dist = 2400
results_walk = there.cluster_index(walk_network, all_pois, poi_dict, poi_params['weight'], poi_params['distance_constant'],poi_params['num_pois'],
                             poi_params['diminishing_returns_constant'],poi_variables=poi_params['variable'],distance=maximum_dist)

Finished category: employment
Maximum score: 10.486671343882943 out of 31.9
Finished category: education
Maximum score: 5.583369841431411 out of 14.3
Finished category: shopping
Maximum score: 10.549515367956698 out of 21.6
Finished category: errands
Maximum score: 4.268511016200598 out of 8.9
Finished category: recreation
Maximum score: 11.483773542936062 out of 23.3
loop: 0
Finished category: employment
Maximum score: 12.400349057686912 out of 31.9
Finished category: education
Maximum score: 6.422506380677627 out of 14.3
Finished category: shopping
Maximum score: 12.607254520202057 out of 21.6
Finished category: errands
Maximum score: 5.110533474978783 out of 8.9
Finished category: recreation
Maximum score: 13.712359813717104 out of 23.3
loop: 1


## Calculate index for public transport
For public transport, different functions are needed in order to correctly join POI locations to street-level nodes only. Two networks must be used - an integrated public transport & walking network prepared by UrbanAccess, and a walking only network.

In [11]:
#gtfs_loc = ("Adelaide_gtfs_2.5.2025")
gtfs_loc = "C:\\Users\\z3258367\\OneDrive - UNSW\\#PhD\\Walkability\\WalkTHERE\\there\\examples\\Adelaide_gtfs_2.5.2025"
transit_net = there.prepare_transit_net(place_gdf, proj_crs, gtfs_loc, walk_network=walk_network)

Checking GTFS text file header whitespace... Reading files using encoding: utf-8 set in configuration.
GTFS text file header whitespace check completed. Took 0.18 seconds
--------------------------------
Processing GTFS feed: Adelaide_gtfs_2.5.2025


c:\Users\z3258367\Anaconda3\envs\ox_ua_there\lib\site-packages\urbanaccess\gtfs\utils_format.py:1066: FutureWarning: The default value of regex will change from True to False in a future version.
  col_snake_case = col.str.replace(r'\s+', '_')


GTFS feed: Adelaide_gtfs_2.5.2025, calendar_dates uses the same service_id across multiple agency_ids. This feed calendar_dates table will be modified from its original format to provide service_ids for each agency using a one to many join
GTFS feed: Adelaide_gtfs_2.5.2025, calendar uses the same service_id across multiple agency_ids. This feed calendar table will be modified from its original format to provide service_ids for each agency using a one to many join


c:\Users\z3258367\Anaconda3\envs\ox_ua_there\lib\site-packages\urbanaccess\gtfs\utils_format.py:1066: FutureWarning: The default value of regex will change from True to False in a future version.
  col_snake_case = col.str.replace(r'\s+', '_')
c:\Users\z3258367\Anaconda3\envs\ox_ua_there\lib\site-packages\urbanaccess\gtfs\utils_format.py:1066: FutureWarning: The default value of regex will change from True to False in a future version.
  col_snake_case = col.str.replace(r'\s+', '_')
c:\Users\z3258367\Anaconda3\envs\ox_ua_there\lib\site-packages\urbanaccess\gtfs\utils_format.py:1066: FutureWarning: The default value of regex will change from True to False in a future version.
  col_snake_case = col.str.replace(r'\s+', '_')


GTFS feed: Adelaide_gtfs_2.5.2025, stops uses the same stop_id across multiple agency_ids. This feed stops table will be modified from its original format to provide stop_ids for each agency using a one to many join


...

agency.txt agency_name column has more than one agency name listed. Unique agency id was assigned using the agency id and associated agency name.
Unique agency id operation complete. Took 4.31 seconds
Unique GTFS feed id operation complete. Took 0.00 seconds
Records:
     stop_id            unique_agency_id  stop_code  \
306     5488  school_service_-_southlink    15338.0   
307     5488                   southlink    15338.0   
308     5516  school_service_-_southlink    15366.0   
309     5516                   southlink    15366.0   
310     7304                   southlink    16670.0   
...      ...                         ...        ...   
1189  100787                   southlink    17882.0   
1190  100790                   southlink    17885.0   
1191  100792                   southlink    17887.0   
1253  101272                   southlink    18363.0   
4155  101237             torrens_transit    18328.0   

                              stop_name                        stop_des

c:\Users\z3258367\Anaconda3\envs\ox_ua_there\lib\site-packages\urbanaccess\gtfs\network.py:744: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for trip, tmp_trip_df in stop_times_df.groupby(['unique_trip_id']):


stop time table transformation to Pandana format edge table completed. Took 15.68 seconds
Time conversion completed: seconds converted to minutes.
9,304 of 9,697 records selected from stops. Took 0.04 seconds
stop time table transformation to Pandana format node table completed. Took 0.01 seconds
route type successfully joined to transit edges. Took 3.07 seconds
route id successfully joined to transit edges. Took 0.20 seconds
Successfully created transit network. Took 19.90 seconds
Stop times from 07:00:00 to 12:00:00 successfully selected 137,469 records out of 160,361 total records (85.72 percent of total). Took 0.05 seconds
Starting route stop headway calculation for 20,479 route stops...
Route stop headway calculation complete. Took 128.32 seconds
headway calculation complete. Took 132.85 seconds
Created OSM network with travel time impedance using a travel speed of 1.875 MPH. Took 0.03 seconds
Loaded UrbanAccess network components comprised of:
     Transit: 9,304 nodes and 133,50

c:\Users\z3258367\Anaconda3\envs\ox_ua_there\lib\site-packages\geopandas\array.py:1406: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  warnings.warn(


Edge and node tables formatted for Pandana with integer node ids: id_int, to_int, and from_int. Took 5.87 seconds
Network edge and node network integration completed successfully resulting in a total of 170,317 nodes and 736,617 edges:
     Transit: 20,478 nodes 133,507 edges;
     OSM: 149,839 nodes 445,164 edges; and
     OSM/Transit connector: 157,946 edges.


In [12]:
transit_net_pandana, walk_transit_net_pandana = there.pandana_transit_net(transit_net, place_gdf)

In [15]:
transit_index = there.transit_cluster_index(walk_transit_net_pandana, transit_net_pandana, all_pois, poi_dict, poi_params['weight'], poi_params['distance_constant'],
                                poi_params['num_pois'], poi_params['diminishing_returns_constant'], poi_variables=poi_params['variable'],
                                distance=90)

Finished category: employment
Maximum score: 1.7125164852478252 out of 31.9
Finished category: education
Maximum score: 10.587841345549155 out of 14.3
Finished category: shopping
Maximum score: 15.992823291179144 out of 21.6
Finished category: errands
Maximum score: 6.589635522754369 out of 8.9
Finished category: recreation
Maximum score: 17.2515177168738 out of 23.3
loop: 0
Finished category: employment
Maximum score: 1.4392806803041156 out of 31.9
Finished category: education
Maximum score: 8.898527766621028 out of 14.3
Finished category: shopping
Maximum score: 13.441132850280711 out of 21.6
Finished category: errands
Maximum score: 5.5382445540508485 out of 8.9
Finished category: recreation
Maximum score: 14.498999787571323 out of 23.3
loop: 1


In [50]:
wgs_pois = all_pois.to_crs("EPSG:4326")

In [51]:
transit_index = there.transit_index(walk_transit_net_pandana, transit_net_pandana, wgs_pois, poi_dict, poi_params['weight'], poi_params['distance_constant'],
                                poi_params['num_pois'], poi_params['diminishing_returns_constant'], poi_variables=poi_params['variable'],
                                distance=90)

Finished category: employment
Maximum score: 29.586309423153022 out of 31.9
Finished category: education
Maximum score: 10.545235860232033 out of 14.3
Finished category: shopping
Maximum score: 15.989382850308775 out of 21.6
Finished category: errands
Maximum score: 6.585686117317276 out of 8.9
Finished category: recreation
Maximum score: 17.251103516569348 out of 23.3
